# Presentation of gathered results


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import math
import pandas as pandas
from pandas import concat
import numpy as np
from scipy.stats import pearsonr
from statistics import mean
from numpy.random import shuffle

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

data = pandas.read_json("../data/water_loss_data_set_1.json")
data = data[(data.timeStamp != "0000-00-00 00:00:00" ) & (data.timeStamp != "2000-01-01 00:00:00")]
data = data[(data.tot1 != 0 ) & (data.analog2 != 0)]
data["timeStamp"] = pandas.to_datetime(data["timeStamp"], format="%Y-%m-%d %H:%M")

data_with_minutes = data.copy(deep=True)

# Normalisation of pressure data
# data["analog2"] = ((data["analog2"] - data["analog2"].min()) / (data["analog2"].max() - data["analog2"].min()))

data_249_min = data[data['idflowmeter'] == "MAG8000_024905H318"]
data_248_min = data[data['idflowmeter'] == "MAG8000_024805H318"]
data_249_min

In [ ]:
data

In [ ]:
def data_sel_time(start, end, data_fun):
    return data_fun.loc[(data_fun["timeStamp"] >= start) & (data_fun["timeStamp"] <= end)]   

def derivate(x_data, y_data):
    x_arr = []
    y_arr = []
    derivate_y = np.diff(y_data, n=2) # second derivative
    idx_max_dy = np.argmax(derivate_y)

    """    # initialize N  
    N = 20  # Indices of N largest elements in list 
    res = sorted(range(len(derivate_y)), key = lambda sub: derivate_y[sub])[-N:][::-1] """
    # print(sorted(set(derivate_y)))
    
    for index, value in enumerate(derivate_y):
        if abs(value) >= 0.05:
            # print(np.datetime_as_string(x_data[index], unit='m'), " value:", y_data[index])
            x_arr.append(x_data[index])
            y_arr.append(y_data[index])
    
    return x_arr, y_arr

### Overview of all data with an option to zoom on specific part 

List of dates where an anomaly occurs:
- 24.1.2019 7.27
- 25.1.2019 6.25

In [ ]:
# anomaly_rows_248 = data_248_min.loc[(data_248_min['analog2'] <= 0.72)]
data_249_processed = data_sel_time("2018-11-28 08:00:00", "2020-06-26 08:00:00", data_249_min)
anomaly_rows_249 = data_249_processed.loc[(data_249_processed['analog2'] <= 0.72)]
dates_249 = anomaly_rows_249["timeStamp"].map(pandas.Timestamp.date).unique()
dates_249 = [i.strftime("%d-%m-%Y") for i in dates_249]

with pandas.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(dates_249)

Graph with visualization of all data

In [ ]:
df = data_sel_time("2018-11-20 08:00:00", "2020-06-26 08:00:00", data_249_min)
x = df["timeStamp"].values
y = df["analog2"].values
# print(x)

"""fig = px.line(df, x="timeStamp", y="analog2", height=600)
fig.show()"""

Graph with criticical points they are only determined after an anomaly already happends

In [ ]:
x_d, y_d = derivate(x, y)

"""
# With "critical points"
fig = go.Figure()
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.add_trace(go.Scatter(x=x_d, y=y_d, mode='markers', name='markers'))
""";

Prasi kaj je blo za te datume:  
['09-01-2019', '10-01-2019', '13-01-2019', '24-01-2019', '25-01-2019', '26-01-2019', '12-03-2019', '31-03-2019', '04-04-2019', '05-04-2019', '09-04-2019', '14-04-2019', '15-04-2019', '17-04-2019', '27-04-2019', '04-05-2019', '06-05-2019', '12-05-2019', '13-05-2019', '14-05-2019', '15-05-2019', '01-06-2019', '05-06-2019', '08-06-2019', '10-06-2019', '12-06-2019', '13-06-2019', '15-06-2019', '17-06-2019', '18-06-2019', '19-06-2019', '22-06-2019', '24-06-2019', '03-08-2019', '28-08-2019', '30-09-2019', '03-10-2019', '31-10-2019', '27-11-2019', '18-12-2019', '30-12-2019', '08-01-2020', '05-02-2020', '27-02-2020']
  
   
Also kaj je blo med 24.1.2019 in 4.2.2019 in med aprilom in oktobrom v obeh primerih so vrhi nizji. Poglej si Isolation Forest

In [ ]:
https://scikit-learn.org/stable/auto_examples/ensemble/plot_isolation_forest.html
from sklearn.ensemble import IsolationForest

rng = np.random.RandomState(42)

# Generate train data
X = 0.3 * rng.randn(100, 2)
X_train = np.r_[X + 2, X - 2]
# Generate some regular novel observations
X = 0.3 * rng.randn(20, 2)
X_test = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))

# fit the model
clf = IsolationForest(max_samples=100, random_state=rng)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)

In [ ]:
"""
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='lines+markers',
                    name='lines+markers'))

fig.show()""";